#Recommender system

##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

##Mount Google Drive / Load datasets

In [43]:
from google.colab import drive
drive.mount('/content/drive')

# !ls /content/drive/MyDrive : For checking the files

# Define the file paths for movies.dat and users.dat
movies_file_path = '/content/drive/MyDrive/movies.dat'
ratings_file_path = '/content/drive/MyDrive/ratings.dat'
users_file_path = '/content/drive/MyDrive/users.dat'

#Load datasets
training_set = pd.read_csv('/content/drive/MyDrive/u1.base', delimiter = '\t', header = None)
test_set = pd.read_csv('/content/drive/MyDrive/u1.test', delimiter = '\t', header = None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Convert to NumPy arrays

In [44]:
training_set = training_set.iloc[:, :3].to_numpy()
test_set = test_set.iloc[:, :3].to_numpy()

#Recalculate the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print(nb_users, nb_movies)


943 1682


##Get the number of users and movies

In [45]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print(nb_users, nb_movies)

943 1682


##Convert function

In [46]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users].astype(int)
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        if len(id_movies) > 0:
            ratings[id_movies - 1] = id_ratings
        new_data.append(ratings)
    return np.array(new_data)


##Convert datasets

In [47]:
training_set = convert(training_set)
test_set = convert(test_set)

##Converting ratings to pytorch tensors

In [48]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

##Converting the ratings into binary ratings 1 (liked) or 0 (not liked)

In [49]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1


##Set up the RBM architecture

In [72]:
class RBM(nn.Module):
    def __init__(self, n_visible, n_hidden):
        super(RBM, self).__init__()
        # Initialize weights and biases
        self.W = nn.Parameter(torch.randn(n_hidden, n_visible) * 0.1)  # Weights
        self.a = nn.Parameter(torch.zeros(1, n_hidden))  # Bias for hidden layer
        self.b = nn.Parameter(torch.zeros(1, n_visible))  # Bias for visible layer

    # Sample hidden nodes given visible nodes
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t()) + self.a
        prob_h_given_v = torch.sigmoid(wx)  # Probability of hidden node being activated
        return prob_h_given_v, torch.bernoulli(prob_h_given_v)

    # Sample visible nodes given hidden nodes
    def sample_v(self, y):
        wy = torch.mm(y, self.W) + self.b
        prob_v_given_h = torch.sigmoid(wy)  # Probability of visible node being activated
        return prob_v_given_h, torch.bernoulli(prob_v_given_h)

    # Forward pass: Gibbs Sampling
    def forward(self, x):
        prob_h_given_v, h_sample = self.sample_h(x)
        prob_v_given_h, v_sample = self.sample_v(h_sample)
        return prob_v_given_h, v_sample


##Initialize the RBM

In [73]:
#Set the number of neurons in the visible and hidden layers

n_visible = nb_movies #Number of movies (visible layer size)
n_hidden = 100 #Number of hidden neurons (customizable)

rbm = RBM(n_visible, n_hidden)

#Check that the RBM methods are working
print(rbm.sample_h)
print(rbm.sample_v)

<bound method RBM.sample_h of RBM()>
<bound method RBM.sample_v of RBM()>


##Defining the Training Process

In [76]:
#Training process involves reconstructing the data and optimizing the weights and biases
#contrastive divergence used for training

def train_rbm(rbm, data, epochs=10, batch_size=64, learning_rate=0.01):
    optimizer = torch.optim.SGD([rbm.W, rbm.a, rbm.b], lr=learning_rate)

    for epoch in range(epochs):
        train_loss = 0
        s = 0.0

        for i in range(0, len(data) - batch_size, batch_size):
            batch = data[i:i + batch_size]
            v0 = batch
            # Forward pass (Gibbs Sampling)
            prob_h_given_v, h_sample = rbm.sample_h(v0)
            prob_v_given_h, v_sample = rbm.sample_v(h_sample)

            # Compute the loss (reconstruction error)
            v1 = v_sample
            h1, _ = rbm.sample_h(v1)

            # Update weights and biases
            optimizer.zero_grad()
            rbm.W.grad = torch.mm(h_sample.t(), v0) - torch.mm(h1.t(), v1)
            rbm.a.grad = torch.sum(h_sample - h1, 0, keepdim = True) #preserve the dimension
            rbm.b.grad = torch.sum(v0 - v1, 0, keepdim = True) #preserve the dimension

            optimizer.step()

            # Compute reconstruction loss
            train_loss += torch.mean(torch.abs(v0[v0 >= 0] - v1[v0 >= 0]))
            s += 1.0

        print(f'Epoch: {epoch + 1}, Loss: {train_loss / s}')



##Train the RBM

In [77]:
train_rbm(rbm, training_set, epochs=10, batch_size=64, learning_rate=0.01)



Epoch: 1, Loss: 0.22065843641757965
Epoch: 2, Loss: 0.1749064028263092
Epoch: 3, Loss: 0.1749064028263092
Epoch: 4, Loss: 0.1749064028263092
Epoch: 5, Loss: 0.1749064028263092
Epoch: 6, Loss: 0.1749064028263092
Epoch: 7, Loss: 0.1749064028263092
Epoch: 8, Loss: 0.1749064028263092
Epoch: 9, Loss: 0.1749064028263092
Epoch: 10, Loss: 0.1749064028263092


##Test the RBM

In [78]:
def test_rbm(rbm, training_set, test_set):
  test_loss = 0
  s = 0.0

  for i in range(len(test_set)):
    v = training_set[i:i + 1]
    vt = test_set[i:i + 1]
    if len(vt[vt >= 0]) > 0: #Avoid missing ratings
      _, h = rbm.sample_h(v)
      prob_v, _ = rbm.sample_v(h)
      test_loss += torch.mean(torch.abs(vt[vt >= 0] - prob_v[vt >= 0]))
      s += 1.0

  print(f'Test Loss: {test_loss / s}')

##Run the test

In [79]:
test_rbm(rbm, training_set, test_set)

#NOTE ON NEXT STEPS: Fine-tune the hyperparameters:
#- Increase the number of epochs
#- Decrease the learning rate
#- Increase the batch size

#Analyze the results to understand the performance of the RBM
#compare the predictions with the actual ratings to evaluate the model

Test Loss: 0.15892033278942108


##Save the model

In [80]:
torch.save(rbm.state_dict(), 'rbm_model.pth')